In [1]:
!pip3 install --upgrade langchain langchain_experimental openai --quiet 
!pip3 install --upgrade matplotlib --quiet
!pip3 install --upgrade pandas --quiet
!pip3 install --upgrade sqlalchemy --quiet


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import os
import csv
import pandas as pd
from datetime import datetime
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
import sqlalchemy

In [3]:
# initialise sql engine
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/full_iseller_data.db')

In [4]:
def WeekOfMonth(date):
    first_day_of_month = date.replace(day=1)
    day_of_week_first_day = first_day_of_month.weekday()
    week_number = (date.day + day_of_week_first_day - 1) // 7 + 1
    return week_number

In [5]:
# import csv and convert to db
iseller_data = pd.read_excel('/Users/davidrinaldi/Documents/Projects/myk-dashboard/data/full_iseller_data.xlsx')
# add order date data
iseller_data['closed_date'] = pd.to_datetime(iseller_data['closed_date'])
iseller_data['date'] = iseller_data['closed_date'] .dt.date
iseller_data['day_of_week_num'] = iseller_data['closed_date'].dt.dayofweek
iseller_data['day_name'] = iseller_data['closed_date'].dt.day_name()
iseller_data['day'] = iseller_data['closed_date'].dt.day
iseller_data['month'] = iseller_data['closed_date'].dt.month
iseller_data['week'] = iseller_data['closed_date'].dt.isocalendar().week
iseller_data['year'] = iseller_data['closed_date'].dt.year

In [6]:
#db = SQLDatabase.from_uri("sqlite:///data/full_iseller_data.db")


In [7]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI, OpenAI

query_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)
analysis_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

## location table

In [8]:
# initialise location tables
from scripts.utils import generate_location_df

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp.db')
location_df = generate_location_df(iseller_data)
location_df.to_sql('locations', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp.db")


In [9]:
location_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [10]:
response = location_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


locations
Invoking: `sql_db_schema` with `{'table_names': 'locations'}`



CREATE TABLE locations (
	outlet_name TEXT, 
	year INTEGER, 
	month_name TEXT, 
	month INTEGER, 
	quantity BIGINT, 
	quantity_diff FLOAT, 
	growth_rate FLOAT
)

/*
3 rows from locations table:
outlet_name	year	month_name	month	quantity	quantity_diff	growth_rate
Ayam! Alun-Alun BKS	2024	March	3	172	None	None
Ayam! Alun-Alun BKS	2024	April	4	134	-38.0	-22.093023255813954
Ayam! Alun-Alun BKS	2024	May	5	23	-111.0	-82.83582089552239
*/1. What is the total quantity sold for each outlet in a specific year?
2. How does the quantity sold in each outlet vary month by month?
3. Can you provide the top outlets with the highest growth rate of quantity sold? <EOF>

> Finished chain.
1. What is the total quantity sold for each outlet in a specific year?
2. How does the quantity sold in each outlet vary month by month?
3. Can you provide th

In [11]:
location_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
location_questions

['1. What is the total quantity sold for each outlet in a specific year?',
 '2. How does the quantity sold in each outlet vary month by month?',
 '3. Can you provide the top outlets with the highest growth rate of quantity sold? ']

In [12]:
# create location answer model
location_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [13]:
location_responses = []
for question in location_questions:
    print(question)
    response = location_query.invoke(question + "All prices are in IDR")
    location_responses.append(response['output'])


1. What is the total quantity sold for each outlet in a specific year?
2. How does the quantity sold in each outlet vary month by month?
3. Can you provide the top outlets with the highest growth rate of quantity sold? 


In [14]:
print(location_responses[0])
    

The total quantity sold for each outlet in the year 2024, in descending order, is as follows:

1. Bekasi - Perjuangan: 39,876 units
2. Jaksel - Pondok Indah: 14,574 units
3. Cibubur: 14,186 units
4. Jakut - Kelapa Gading: 10,761 units
5. Jakbar - Taman Palem: 8,531 units
6. Tangerang - Pisa Grande: 6,826 units
7. Bandung: 2,264 units
8. Ayam! Dapur Kitchen BKS: 1,929 units
9. Ayam! RSIJ: 412 units
10. Ayam! Libersa Park BKS: 347 units

These are the total quantities sold for each outlet in the specified year.


In [15]:
print(location_responses[1])


Here are some examples of how the quantity sold in each outlet varies month by month:

1. Outlet: Ayam! Alun-Alun BKS, Month: March, Quantity Sold: 172
2. Outlet: Ayam! Alun-Alun BKS, Month: April, Quantity Sold: 134
3. Outlet: Ayam! Alun-Alun BKS, Month: May, Quantity Sold: 23
4. Outlet: Ayam! Dapur Kitchen BKS, Month: January, Quantity Sold: 366
5. Outlet: Ayam! Dapur Kitchen BKS, Month: February, Quantity Sold: 331
6. Outlet: Ayam! Dapur Kitchen BKS, Month: March, Quantity Sold: 524
7. Outlet: Ayam! Dapur Kitchen BKS, Month: April, Quantity Sold: 662
8. Outlet: Ayam! Dapur Kitchen BKS, Month: May, Quantity Sold: 46
9. Outlet: Ayam! Libersa Park BKS, Month: January, Quantity Sold: 121
10. Outlet: Ayam! Libersa Park BKS, Month: February, Quantity Sold: 138

These are just a few examples of how the quantity sold varies month by month for different outlets.


In [16]:
print(location_responses[2])


The top outlets with the highest growth rate of quantity sold are:

1. Ayam! RSIJ - Growth Rate: 375.0
2. Ayam! Dapur Kitchen BKS - Growth Rate: 58.31
3. Jakbar - Taman Palem - Growth Rate: 40.56
4. Jaksel - Pondok Indah - Growth Rate: 32.15
5. Bandung - Growth Rate: 28.21
6. Ayam! Dapur Kitchen BKS - Growth Rate: 26.34
7. Cibubur - Growth Rate: 16.89
8. Ayam! Libersa Park BKS - Growth Rate: 14.05
9. Bekasi - Perjuangan - Growth Rate: 7.25
10. Jakut - Kelapa Gading - Growth Rate: 6.36


In [17]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/location.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(location_questions[idx]), 
                escape_newlines(location_responses[idx])
            ]
        )

# SALES

## sales table

In [18]:
# initialise sales tables
from scripts.utils import generate_sales_table

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp2.db')
sales_df = generate_sales_table(iseller_data)
sales_df.to_sql('sales', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp2.db")

In [19]:
sales_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [20]:
response = sales_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


sales
Invoking: `sql_db_schema` with `{'table_names': 'sales'}`



CREATE TABLE sales (
	clean_product_name TEXT, 
	product_type TEXT, 
	total_revenue BIGINT, 
	current_quarter_revenue FLOAT, 
	last_quarter_revenue FLOAT, 
	current_month_revenue FLOAT, 
	last_month_revenue FLOAT, 
	quarterly_growth FLOAT, 
	monthly_growth FLOAT, 
	"quarter_Growth(%)" FLOAT, 
	"Monthly_Growth(%)" FLOAT
)

/*
3 rows from sales table:
clean_product_name	product_type	total_revenue	current_quarter_revenue	last_quarter_revenue	current_month_revenue	last_month_revenue	quarterly_growth	monthly_growth	quarter_Growth(%)	Monthly_Growth(%)
(es batu) crystalline cup	REGULAR	201033098	55569433.0	145463665.0	10521000.0	45048433.0	-61.79840993281724	-76.64513658000047	-0.45	-0.62
a&w root beer kaleng	REGULAR	21235100	5086300.0	16148800.0	89000.0	4997300.0	-68.50354205885267	-98.21903828067157	-0.52	-0.97
absolut vodka	VODKA	515999

In [21]:
sales_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
sales_questions

['1. What is the distribution of product types in the sales data?',
 '2. How does the current quarter revenue compare to the last quarter revenue for different product types?',
 '3. What is the average monthly growth percentage across all products?  ']

In [22]:
# create location answer model
sales_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [23]:
sales_responses = []
for question in sales_questions:
    print(question)
    response = sales_query.invoke(question + "All prices are in IDR")
    sales_responses.append(response['output'])

1. What is the distribution of product types in the sales data?
2. How does the current quarter revenue compare to the last quarter revenue for different product types?
3. What is the average monthly growth percentage across all products?  


In [24]:
print(sales_responses[0])

The distribution of product types in the sales data is as follows:

- ANGGUR: 19
- BEER: 61
- COGNAC: 5
- EMBALASE: 2
- FOOD: 46
- FREE: 1
- GIN: 11
- LIQUOR: 46
- MAKGEOLLI: 5
- MERCHANDISE: 12
- MEZCAL: 1
- REGULAR: 31
- ROKOK: 8
- RUM: 14
- SOJU: 76
- SPIRIT: 9
- TEQUILA: 1
- VAPE: 4
- VODKA: 34
- WHISKY: 45
- WINE: 106


In [25]:
print(sales_responses[1])

The current quarter revenue compared to the last quarter revenue for different product types are as follows:

1. Product Type: REGULAR
   - Current Quarter Revenue: 55,569,433 IDR
   - Last Quarter Revenue: 145,463,665 IDR

2. Product Type: REGULAR
   - Current Quarter Revenue: 5,086,300 IDR
   - Last Quarter Revenue: 16,148,800 IDR

3. Product Type: VODKA
   - Current Quarter Revenue: 18,173,000 IDR
   - Last Quarter Revenue: 33,426,900 IDR

4. Product Type: ANGGUR
   - Current Quarter Revenue: 40,367,800 IDR
   - Last Quarter Revenue: 142,327,100 IDR

5. Product Type: ANGGUR
   - Current Quarter Revenue: 18,252,100 IDR
   - Last Quarter Revenue: 52,550,000 IDR

6. Product Type: VODKA
   - Current Quarter Revenue: 3,418,800 IDR
   - Last Quarter Revenue: 2,600,100 IDR

7. Product Type: VODKA
   - Current Quarter Revenue: 2,655,000 IDR
   - Last Quarter Revenue: 5,478,800 IDR

8. Product Type: SOJU
   - Current Quarter Revenue: 12,978,100 IDR
   - Last Quarter Revenue: 24,177,000 IDR



In [26]:
print(sales_responses[2])

I'm sorry, but it seems there was an issue with retrieving the average monthly growth percentage across all products. The query did not return any results.


In [27]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/sales.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(sales_questions[idx]), 
                escape_newlines(sales_responses[idx])
            ]
        )

## top product by profit

In [31]:
# Top Product by profit
product_quantity = iseller_data.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['profit'].sum().reset_index()\
        .sort_values(by='profit', ascending=False)

In [32]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp7.db')
product_quantity.to_sql('top_product_by_profit', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp7.db")

In [33]:
product_quantity_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [34]:
response = product_quantity_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_profit
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_profit'}`



CREATE TABLE top_product_by_profit (
	year INTEGER, 
	month INTEGER, 
	week BIGINT, 
	day_name TEXT, 
	day_of_week_num INTEGER, 
	clean_product_name TEXT, 
	profit FLOAT
)

/*
3 rows from top_product_by_profit table:
year	month	week	day_name	day_of_week_num	clean_product_name	profit
2024	2	5	Friday	4	bintang botol	18341206.0
2024	1	2	Friday	4	happy soju original	17331836.0
2024	1	2	Friday	4	wija soju lychee	15659100.0
*/1. What are the top products with the highest profits in each week?
2. How do the profits of different products vary across different days of the week?
3. Can we identify any seasonal trends in product profits based on the month and year data?
<EOF>

> Finished chain.
1. What are the top products with the highest profits in each week?
2. How do the profits of different products vary ac

In [35]:
pq_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
pq_questions

['1. What are the top products with the highest profits in each week?',
 '2. How do the profits of different products vary across different days of the week?',
 '3. Can we identify any seasonal trends in product profits based on the month and year data?']

In [36]:
# create location answer model
pq_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [37]:
import random

pq_responses = []
questions = ['What is the correlation between external events (such as holidays, festivals, or economic changes) and monthly sales patterns? (Move to forecasting)', 'Are there any emerging trends or shifts in consumer preferences over the months and years?', 'How does the performance of different products compare across the same time periods (e.g., year-over-year comparisons for the same month)?']
for question in pq_questions:
    try:
        print(question)
        response = pq_query.invoke(question + "All prices are in IDR")
        pq_responses.append(response['output'])
    except:
        question = random.choice(questions)
        print(question)
        response = pq_query.invoke(question + "All prices are in IDR")
        pq_responses.append(response['output'])
        

1. What are the top products with the highest profits in each week?
2. How do the profits of different products vary across different days of the week?
3. Can we identify any seasonal trends in product profits based on the month and year data?


In [38]:
print(pq_responses[0])

The query results show the top products with the highest profits in each week:
- Week 1: 
  - Product: kawa kawa anggur hijau
  - Profit: 7,761,904 IDR

Please note that the results are limited to the top 10 products with the highest profits in each week. If you would like to see more results or for a specific week, feel free to let me know!


In [39]:
print(pq_responses[1])

The profits of different products vary across different days of the week. Here are the top 10 products with their profits on different days:

1. On Friday, the product "bintang botol" has a profit of 18,341,206 IDR.
2. On Friday, the product "happy soju original" has a profit of 17,331,836 IDR.
3. On Friday, the product "wija soju lychee" has a profit of 15,659,100 IDR.
4. On Friday, the product "anggur merah gold" has a profit of 12,750,050 IDR.
5. On Saturday, the product "kawa kawa anggur hijau" has a profit of 11,733,007 IDR.
6. On Tuesday, the product "kawa kawa anggur hijau" has a profit of 11,724,227 IDR.
7. On Friday, the product "jameson" has a profit of 11,333,299 IDR.
8. On Friday, the product "anggur merah" has a profit of 11,245,250 IDR.
9. On Friday, the product "happy soju lychee" has a profit of 11,145,636 IDR.
10. On Saturday, the product "api anggur hijau" has a profit of 11,049,500 IDR.


In [40]:
print(pq_responses[2])

Based on the data from the "top_product_by_profit" table, here are the total profits for each month in the year 2024:

- January 2024: IDR 2,643,294,790.12
- February 2024: IDR 2,899,852,576.48
- March 2024: IDR 2,111,684,501.98
- April 2024: IDR 2,115,140,338.01
- May 2024: IDR 687,248,785.00

It seems that there are fluctuations in profits across different months in the year 2024.


In [41]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/pq.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(pq_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(pq_questions[idx]), 
                escape_newlines(pq_responses[idx])
            ]
        )

## Top Product by Quantity Sold

In [28]:
# Top Product by Quantity Sold
tpqs = iseller_data.groupby(['clean_product_name', 'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])
tpqs.head()

,clean_product_name,year,month,week,total_amount,quantity
3595,kawa kawa anggur hijau,2024,1,1,140173.195980,557
340,anggur merah gold,2024,1,1,131484.556314,395
457,api anggur hijau,2024,1,1,164322.362245,292
296,anggur merah,2024,1,1,118656.692308,231
1125,bintang kaleng,2024,1,1,222735.696429,224


In [29]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp3.db')
tpqs.to_sql('top_product_by_quantity_sold', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp3.db")

In [30]:
tpqs_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [31]:
response = tpqs_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_quantity_sold
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_quantity_sold'}`



CREATE TABLE top_product_by_quantity_sold (
	clean_product_name TEXT, 
	year BIGINT, 
	month BIGINT, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from top_product_by_quantity_sold table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What are the top selling products based on total amount sold in a specific year?
2. How does the quantity of products sold vary across different months and weeks?
3. Can we identify any trends in the sales of specific product categories over the years?
<EOF>

> Finished chain.
1. What are the top selling products based on total amount sold in a specific year?
2. How does

In [32]:
tpqs_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpqs_questions

['1. What are the top selling products based on total amount sold in a specific year?',
 '2. How does the quantity of products sold vary across different months and weeks?',
 '3. Can we identify any trends in the sales of specific product categories over the years?']

In [33]:
# create location answer model
tpqs_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [34]:
tpqs_responses = []
for question in tpqs_questions:
    print(question)
    response = tpqs_query.invoke(question + "All prices are in IDR")
    tpqs_responses.append(response['output'])

1. What are the top selling products based on total amount sold in a specific year?
2. How does the quantity of products sold vary across different months and weeks?
3. Can we identify any trends in the sales of specific product categories over the years?


In [35]:
print(tpqs_responses[0])

The top selling products based on total amount sold in the year 2024 are as follows (in IDR):
1. Hibiki Japanese Harmony - 5,377,000
2. Hibiki Japanese Harmony - 4,887,000
3. Nikka Whisky from the Barrel Japanese Whisky - 4,545,016
4. Ballantines Blended Scotch Whisky - 4,545,016
5. Jack Daniel's No.7 - 4,259,000
6. Hennessy VSOP - 4,196,000
7. Monkey Shoulder - 3,947,000
8. Sababay Mascetti - 3,944,000
9. Roku Gin - 3,937,000
10. Martell VSOP - 3,832,488


In [36]:
print(tpqs_responses[1])

The query results show the total quantity of products sold across different months and weeks:

- In month 1, week 1: 6597 products sold
- In month 1, week 2: 6022 products sold
- In month 1, week 3: 6099 products sold
- In month 1, week 4: 6025 products sold
- In month 1, week 5: 2134 products sold
- In month 2, week 5: 4417 products sold
- In month 2, week 6: 7422 products sold
- In month 2, week 7: 5808 products sold
- In month 2, week 8: 5497 products sold
- In month 2, week 9: 2347 products sold

This information provides an overview of how the quantity of products sold varies across different months and weeks.


In [37]:
print(tpqs_responses[2])

Based on the sales data from the "top_product_by_quantity_sold" table, we can identify trends in the sales of specific product categories over the year 2024. Here are some examples of total sales for different product categories in IDR:

1. Chivas 12yo - Total Sales in 2024: 22,505,771.37 IDR
2. Bombay Sapphire - Total Sales in 2024: 12,256,864.08 IDR
3. Hennessy VSOP - Total Sales in 2024: 35,864,390.67 IDR
4. Grey Goose - Total Sales in 2024: 21,013,684.14 IDR
5. Jack Daniel's No.7 - Total Sales in 2024: 17,077,432.33 IDR

These examples show the total sales for specific product categories in IDR for the year 2024.


In [38]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpqs.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpqs_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpqs_questions[idx]), 
                escape_newlines(tpqs_responses[idx])
            ]
        )

## top product by category sold

In [39]:
# Top Product by Category Sold
# Get average price of category too 

tpcs = iseller_data.groupby(['product_type',  'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])

In [40]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp4.db')
tpqs.to_sql('top_product_by_category_sold', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp4.db")

In [41]:
tpcs_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [42]:
response = tpcs_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_category_sold
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_category_sold'}`



CREATE TABLE top_product_by_category_sold (
	clean_product_name TEXT, 
	year BIGINT, 
	month BIGINT, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from top_product_by_category_sold table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/- What are the top-selling products by category for the current year?
- How do the sales quantities vary by month for the top products?
- Is there a correlation between total sales amount and quantity sold for the top products?
<EOF>

> Finished chain.
- What are the top-selling products by category for the current year?
- How do the sales quantities vary by month for the t

In [43]:
tpcs_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpcs_questions

['- What are the top-selling products by category for the current year?',
 '- How do the sales quantities vary by month for the top products?',
 '- Is there a correlation between total sales amount and quantity sold for the top products?']

In [44]:
# create location answer model
tpcs_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [45]:
tpcs_responses = []
for question in tpcs_questions:
    print(question)
    response = tpcs_query.invoke(question + "All prices are in IDR")
    tpcs_responses.append(response['output'])

- What are the top-selling products by category for the current year?
- How do the sales quantities vary by month for the top products?
- Is there a correlation between total sales amount and quantity sold for the top products?


In [46]:
print(tpcs_responses[0])

The top-selling products by category for the current year (2024) are as follows (in IDR):
1. Product: Hibiki Japanese Harmony, Total Amount: 5,377,000, Quantity: 1
2. Product: Hibiki Japanese Harmony, Total Amount: 4,887,000, Quantity: 2
3. Product: Nikka Whisky from the Barrel Japanese Whisky, Total Amount: 4,545,016, Quantity: 1
4. Product: Ballantines Blended Scotch Whisky, Total Amount: 4,545,016, Quantity: 1
5. Product: Jack Daniel's No.7, Total Amount: 4,259,000, Quantity: 1
6. Product: Hennessy VSOP, Total Amount: 4,196,000, Quantity: 4
7. Product: Monkey Shoulder, Total Amount: 3,947,000, Quantity: 2
8. Product: Sababay Mascetti, Total Amount: 3,944,000, Quantity: 4
9. Product: Roku Gin, Total Amount: 3,937,000, Quantity: 1
10. Product: Martell VSOP, Total Amount: 3,832,488, Quantity: 6


In [47]:
print(tpcs_responses[1])

The top products with the highest sales quantities by month are as follows:

1. Product: kawa kawa anggur hijau, Month: 3, Quantity: 624
2. Product: kawa kawa anggur hijau, Month: 1, Quantity: 557
3. Product: kawa kawa anggur hijau, Month: 2, Quantity: 534
4. Product: kawa kawa anggur hijau, Month: 2, Quantity: 520
5. Product: kawa kawa anggur hijau, Month: 4, Quantity: 455
6. Product: (es batu) crystalline cup, Month: 2, Quantity: 437
7. Product: kawa kawa anggur hijau, Month: 2, Quantity: 435
8. Product: kawa kawa anggur hijau, Month: 3, Quantity: 415
9. Product: kawa kawa anggur hijau, Month: 3, Quantity: 407
10. Product: draft beer kaleng, Month: 2, Quantity: 399

These are the top products with the highest sales quantities for different months.


In [48]:
print(tpcs_responses[2])

Here are some examples of the correlation between total sales amount and quantity sold for the top products:

- Product: Hibiki Japanese Harmony, Total Amount: 5,377,000 IDR, Quantity Sold: 1
- Product: Hibiki Japanese Harmony, Total Amount: 4,887,000 IDR, Quantity Sold: 2
- Product: Nikka Whisky from the Barrel Japanese Whisky, Total Amount: 4,545,016 IDR, Quantity Sold: 1
- Product: Ballantines Blended Scotch Whisky, Total Amount: 4,545,016 IDR, Quantity Sold: 1
- Product: Jack Daniel's No.7, Total Amount: 4,259,000 IDR, Quantity Sold: 1
- Product: Hennessy VSOP, Total Amount: 4,196,000 IDR, Quantity Sold: 4
- Product: Monkey Shoulder, Total Amount: 3,947,000 IDR, Quantity Sold: 2
- Product: Sababay Mascetti, Total Amount: 3,944,000 IDR, Quantity Sold: 4
- Product: Roku Gin, Total Amount: 3,937,000 IDR, Quantity Sold: 1
- Product: Martell VSOP, Total Amount: 3,832,488 IDR, Quantity Sold: 6

It seems that there is a variation in the quantity sold for the top products with different to

In [49]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpcs.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpcs_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpcs_questions[idx]), 
                escape_newlines(tpcs_responses[idx])
            ]
        )

## total quantity sold each month

In [50]:
# Total Quantity sold each month
tqm = iseller_data.groupby(["year", "month", "clean_product_name"])['quantity'].sum().reset_index().sort_values(by=["year", "month"])


In [51]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp5.db')
tpqs.to_sql('total_quantity_sold_monthly', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp5.db")

In [52]:
tpm_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [53]:
response = tpm_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


total_quantity_sold_monthly
Invoking: `sql_db_schema` with `{'table_names': 'total_quantity_sold_monthly'}`



CREATE TABLE total_quantity_sold_monthly (
	clean_product_name TEXT, 
	year BIGINT, 
	month BIGINT, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from total_quantity_sold_monthly table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What are the top 5 clean product names with the highest total amount sold?
2. Can you provide the monthly total quantity sold for each year?
3. Is there any trend in the weekly total amount sold for specific product names? 
<EOF>

> Finished chain.
1. What are the top 5 clean product names with the highest total amount sold?
2. Can you provide the monthly total quantity sold

In [54]:
tpm_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpm_questions

['1. What are the top 5 clean product names with the highest total amount sold?',
 '2. Can you provide the monthly total quantity sold for each year?',
 '3. Is there any trend in the weekly total amount sold for specific product names? ']

In [55]:
# create location answer model
tpm_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [56]:
import random

tpm_responses = []
questions = ['What is the correlation between external events (such as holidays, festivals, or economic changes) and monthly sales patterns? (Move to forecasting)', 'Are there any emerging trends or shifts in consumer preferences over the months and years?', 'How does the performance of different products compare across the same time periods (e.g., year-over-year comparisons for the same month)?']
for question in tpm_questions:
    try:
        print(question)
        response = tpm_query.invoke(question + "All prices are in IDR")
        tpm_responses.append(response['output'])
    except:
        question = random.choice(questions)
        print(question)
        response = tpm_query.invoke(question + "All prices are in IDR")
        tpm_responses.append(response['output'])
        

1. What are the top 5 clean product names with the highest total amount sold?
2. Can you provide the monthly total quantity sold for each year?
3. Is there any trend in the weekly total amount sold for specific product names? 


In [57]:
print(tpm_responses[0])

The top 5 clean product names with the highest total amount sold in IDR are:
1. Hennessy VSOP - IDR 35,864,390.67
2. Martell VSOP - IDR 30,219,499.59
3. Hibiki Japanese Harmony - IDR 24,804,258.00
4. Glenfiddich 12yo - IDR 24,730,651.99
5. JW Black Label - IDR 22,612,401.42


In [58]:
print(tpm_responses[1])

The monthly total quantity sold for each year in IDR are as follows:
- For the year 2024:
  - Month 1: 26,877 units
  - Month 2: 25,491 units
  - Month 3: 20,171 units
  - Month 4: 21,317 units
  - Month 5: 6,372 units


In [59]:
print(tpm_responses[2])    

Yes, there is a trend in the weekly total amount sold for specific product names. Here are some examples of the weekly total amount sold for specific product names:

1. Product Name: (es batu) crystalline cup, Week: 1, Weekly Total Amount Sold: 157,670.77 IDR
2. Product Name: (es batu) crystalline cup, Week: 2, Weekly Total Amount Sold: 150,573.82 IDR
3. Product Name: (es batu) crystalline cup, Week: 3, Weekly Total Amount Sold: 164,179.65 IDR
4. Product Name: (es batu) crystalline cup, Week: 4, Weekly Total Amount Sold: 192,236.29 IDR
5. Product Name: (es batu) crystalline cup, Week: 5, Weekly Total Amount Sold: 343,266.93 IDR
6. Product Name: (es batu) crystalline cup, Week: 6, Weekly Total Amount Sold: 201,923.69 IDR
7. Product Name: (es batu) crystalline cup, Week: 7, Weekly Total Amount Sold: 234,708.42 IDR
8. Product Name: (es batu) crystalline cup, Week: 8, Weekly Total Amount Sold: 186,145.41 IDR
9. Product Name: (es batu) crystalline cup, Week: 9, Weekly Total Amount Sold: 276

In [60]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpm.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpm_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpm_questions[idx]), 
                escape_newlines(tpm_responses[idx])
            ]
        )

## total quantity sold each day

In [26]:
# Total Quantity sold each Day
tqd = iseller_data.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['quantity'].sum().reset_index().sort_values(by="day_of_week_num")

In [29]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp6.db')
tqd.to_sql('total_quantity_sold_daily', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp6.db")

In [30]:
tqd_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [44]:
response = tqd_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


total_quantity_sold_daily
Invoking: `sql_db_schema` with `{'table_names': 'total_quantity_sold_daily'}`



CREATE TABLE total_quantity_sold_daily (
	year BIGINT, 
	month BIGINT, 
	week BIGINT, 
	day_name TEXT, 
	day_of_week_num INTEGER, 
	clean_product_name TEXT, 
	quantity BIGINT
)

/*
3 rows from total_quantity_sold_daily table:
year	month	week	day_name	day_of_week_num	clean_product_name	quantity
2024	4	15	Monday	0	cointreau	1
2024	4	14	Monday	0	jw black label	4
2024	4	14	Monday	0	jw red label	3
*/1. What is the total quantity of "cointreau" products sold in the database?
2. How does the quantity of "jw black label" compare to "jw red label" products sold on Mondays?
3. Can you provide the monthly breakdown of total quantity sold for each product in the dataset?
<EOF>

> Finished chain.
1. What is the total quantity of "cointreau" products sold in the database?
2. How does the quantity of "jw bla

In [45]:
tqd_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tqd_questions

['1. What is the total quantity of "cointreau" products sold in the database?',
 '2. How does the quantity of "jw black label" compare to "jw red label" products sold on Mondays?',
 '3. Can you provide the monthly breakdown of total quantity sold for each product in the dataset?']

In [46]:
# create location answer model
tqd_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [47]:
import random

tqd_responses = []
questions = ['What is the correlation between external events (such as holidays, festivals, or economic changes) and monthly sales patterns? (Move to forecasting)', 'Are there any emerging trends or shifts in consumer preferences over the months and years?', 'How does the performance of different products compare across the same time periods (e.g., year-over-year comparisons for the same month)?']
for question in tqd_questions:
    try:
        print(question)
        response = tqd_query.invoke(question + "All prices are in IDR")
        tqd_responses.append(response['output'])
    except:
        question = random.choice(questions)
        print(question)
        response = tqd_query.invoke(question + "All prices are in IDR")
        tqd_responses.append(response['output'])
        

1. What is the total quantity of "cointreau" products sold in the database?
2. How does the quantity of "jw black label" compare to "jw red label" products sold on Mondays?
3. Can you provide the monthly breakdown of total quantity sold for each product in the dataset?
Are there any emerging trends or shifts in consumer preferences over the months and years?


In [48]:
print(tqd_responses[0])

The total quantity of "cointreau" products sold in the database is 126.


In [49]:
print(tqd_responses[1])

The quantity of "jw black label" products sold on Mondays varies, with values such as 1, 2, 3, 4, 6, and 9. 
The quantity of "jw red label" products sold on Mondays also varies, with values such as 1, 2, 3, 4, 5, 8, and 10.


In [50]:
print(tqd_responses[2])

Based on the total quantity of products sold monthly, the top products are as follows:

1. In March 2024, the product "kawa kawa anggur hijau" had the highest total quantity sold with 2064 units.
2. In February 2024, the product "kawa kawa anggur hijau" had the second-highest total quantity sold with 2038 units.
3. In January 2024, the product "anggur merah gold" had a total quantity sold of 1501 units.
4. In January 2024, the product "anggur merah" had a total quantity sold of 1201 units.
5. In February 2024, the product "(es batu) crystalline cup" had a total quantity sold of 1092 units.
6. In April 2024, the product "kawa kawa anggur hijau" had a total quantity sold of 1085 units.
7. In January 2024, the product "kawa kawa anggur hijau" had a total quantity sold of 1073 units.
8. In January 2024, the product "api anggur hijau" had a total quantity sold of 1027 units.
9. In March 2024, the product "bintang botol" had a total quantity sold of 996 units.
10. In March 2024, the product 

In [52]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tqd.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tqd_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tqd_questions[idx]), 
                escape_newlines(tqd_responses[idx])
            ]
        )

# advanced query response examples

In [62]:
print(tpm_query.invoke('Are there any emerging trends or shifts in consumer preferences over the months and years?')['output'])

Based on the data from the `total_quantity_sold_monthly` table, here are some emerging trends in consumer preferences over the past month:

1. The product "kawa kawa anggur hijau" had a total amount of 170459.99 and a quantity of 308 in May 2024.
2. The product "anggur merah gold" had a total amount of 190785.17 and a quantity of 231 in May 2024.
3. The product "bintang kaleng" had a total amount of 219800.71 and a quantity of 179 in May 2024.
4. The product "api anggur hijau" had a total amount of 183370.60 and a quantity of 160 in May 2024.
5. The product "bintang botol" had a total amount of 188461.67 and a quantity of 158 in May 2024.
6. The product "(es batu) crystalline cup" had a total amount of 122248.26 and a quantity of 156 in May 2024.
7. The product "anggur merah" had a total amount of 215704.98 and a quantity of 127 in May 2024.
8. The product "anggur ginseng intisari" had a total amount of 196629.48 and a quantity of 116 in May 2024.
9. The product "iceland vodka" had a t

In [63]:
test = query_llm.invoke("""
Are there any emerging trends or shifts in consumer preferences over the months and years?

Based on the data from the `total_quantity_sold_monthly` table, here are some emerging trends or shifts in consumer preferences over the months and years:

1. In May 2024, the top-selling products were "kawa kawa anggur hijau" with a total amount of 170459.99 and 308 quantity sold.
2. Following that, "anggur merah gold" was also popular in May 2024 with a total amount of 190785.17 and 231 quantity sold.
3. "bintang kaleng" saw an increase in sales in May 2024 with a total amount of 219800.71 and 179 quantity sold.

These insights indicate changes in consumer preferences over the months and years, with certain products gaining popularity.
""")

In [64]:
print(test.content)


4. In June 2024, there was a shift in consumer preferences towards "blueberry juice" which had a total amount of 152720.60 and 289 quantity sold, showing an increase in demand for this product.
5. "jeruk segar" also saw a rise in sales in June 2024, with a total amount of 142978.50 and 284 quantity sold, indicating a preference for fresh fruit juices among consumers.
6. Additionally, "susu sapi murni" experienced a significant increase in sales in June 2024, with a total amount of 132638.75 and 262 quantity sold, showcasing a trend towards dairy products.

These trends suggest that consumer preferences are constantly evolving, with certain products gaining popularity over time while others may see a decline in demand. It is important for businesses to stay updated on these shifts in order to cater to the changing needs and preferences of their customers.


In [65]:
test = sales_query.invoke('''
Show me month growth rates and what the best selling product was. Suggest a marketing strategy to improve sales. Expand on the marketing strategy giving me a quick outline on what to do
''')

In [66]:
print(test['output'])

The best selling product with the highest monthly growth rate is "anggur merah premium" in the "ANGGUR" category. The monthly growth rate is -94.52%.

To suggest a marketing strategy to improve sales, we can focus on promoting this product through targeted marketing campaigns. Here is a quick outline of the marketing strategy:

1. **Targeted Advertising**: Utilize digital marketing channels such as social media advertising and Google Ads to target customers who are interested in premium wines.
   
2. **Promotions and Discounts**: Offer special promotions and discounts on "anggur merah premium" to attract new customers and encourage repeat purchases.

3. **Collaborations**: Partner with influencers or local wine experts to create buzz around the product and reach a wider audience.

4. **Customer Engagement**: Engage with customers through email marketing campaigns, surveys, and feedback collection to understand their preferences and improve the product offering.

5. **Visual Content**: 